# ACCEL IMPLEMENTATION

In [ ]:
# TODO: fix agent position of the agent based on the config_seed in the MyCustomGrid class
# TODO: visualize the progress of the regret in the main_accel_demo function
# TODO: If the teacher reaches the goal, the simulation should stop

In [ ]:
import numpy as np
import wandb
import gymnasium as gym

import torch

from gymnasium.spaces import Discrete, Box
import gymnasium.spaces as spaces

from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Goal, Wall
from minigrid.minigrid_env import MiniGridEnv, Grid

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_vec_env
import matplotlib.pyplot as plt

# ====================================================
# 1. Custom MiniGrid Environment that returns only the image
#    for SB3's PPO (which expects a Box space).
# ====================================================
class MyCustomGrid(MiniGridEnv):
    """
    Simple MiniGrid environment that places random wall tiles
    according to a config dict, returning only the 'image' observation.
    """

    def __init__(self, config=None, **kwargs):
        if config is None:
            config = {}
        self.config = config

        # Extract parameters from config
        self.width = config.get("width")
        self.height = config.get("height")
        self.custom_seed = config.get("seed_val")
        

        grid_size = max(self.width, self.height)

        mission_space = MissionSpace(mission_func=lambda: "get to the green goal square")

        super().__init__(
            grid_size=grid_size,
            max_steps=self.width * self.height * 2, # max_steps is typically 2x the grid size
            see_through_walls=False,
            agent_view_size=5,                      # Size of the agent's view square
            mission_space=mission_space,
            **kwargs
        )

        # Manually define our observation_space as a single Box (the image).
        # By default, MiniGrid's image shape is (view_size, view_size, 3) if using partial obs,
        # or (height, width, 3) if using full-grid observation. We'll do full-grid here:
        # We'll define (self.height, self.width, 3) as the shape.
        # In practice, "image" shape can vary if partial observations are used.
        self.observation_space = Box(
            low=0,
            high=255,
            shape=(self.agent_view_size, self.agent_view_size, 3),
            dtype=np.uint8
        )

    def _gen_grid(self, width, height):
        """
        Generate the grid layout for a new episode.
        We use self.width and self.height from config, even though the underlying
        MiniGrid environment might use grid_size for some of its operations.
        """    
        
        # Create an empty grid of the "true" width x height from config
        self.grid = Grid(self.width, self.height)
        # Surround the grid with walls
        self.grid.wall_rect(0, 0, self.width, self.height)
        
        # Seed the random number generator to create the same level each time given the same seed
        rng = np.random.default_rng(seed=self.custom_seed)
        
        # Determine the maximum number of blocks (leaving at least one cell for the agent and one for the goal)
        max_blocks = ((self.width - 1) * (self.height - 1)) / 2
        self.num_blocks = rng.integers(1, max_blocks)

        # Place random walls inside using the custom seed. Only place a wall if the cell is empty.
        for _ in range(self.num_blocks):
            r = rng.integers(1, self.height - 1)
            c = rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.put_obj(Wall(), c, r)

        # Place the agent in a random position not occupied by any wall
        while True:
            r = rng.integers(1, self.height - 1)
            c = rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.place_agent(top=(c, r), rand_dir=True)
                break

        # Place the goal object in a random position not occupied by any wall or the agent
        while True:
            r = rng.integers(1, self.height - 1)
            c = rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.put_obj(Goal(), c, r)
                break
            

    def reset(self, **kwargs):
        """
        Override reset to ensure we only return the 'image' array
        instead of a dict with 'image' and 'mission'.
        """
        obs, info = super().reset(**kwargs)
        obs = self._convert_obs(obs)
        return obs, info

    def step(self, action):
        """
        Same for step: override to convert the dict observation into an image only.
        """
        obs, reward, done, truncated, info = super().step(action)
        obs = self._convert_obs(obs)
        return obs, reward, done, truncated, info

    def _convert_obs(self, original_obs):
        """
        original_obs is typically {'image':..., 'mission':...}.
        We'll just return original_obs['image'] to get a Box(low=0,high=255) shape.
        """
        return original_obs["image"]


# ====================================================
# 2. Simple “level buffer” 
# ====================================================
# class to memorize generated levels and score
class LevelBuffer: 
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []  # will store (config_dict, score)

    def add(self, config, score):
        self.data.append((config, score))
        if len(self.data) > self.max_size:
            self.data.sort(key=lambda x: x[1], reverse=True)
            self.data = self.data[: self.max_size]
            #it memorize only the highest score for each level

    def sample_config(self): 
        # Samples a level from the buffer, weighting the probabilities 
        # based on the scores.
        if len(self.data) == 0:
            return None
        scores = [item[1] for item in self.data]
        total = sum(scores)
        if total <= 1e-9:
            # fallback to uniform
            idx = np.random.randint(len(self.data))
            return self.data[idx][0]
        probs = [s / total for s in scores]
        idx = np.random.choice(len(self.data), p=probs)
        return self.data[idx][0]

# ====================================================
# 3. Utility Functions
# ====================================================
def random_config(grid_size):
    return {
        "width": grid_size,
        "height": grid_size,
        "seed_val": np.random.randint(0, 999999),
        # "num_blocks": 3,  # to be set in the MyCustomGrid class depending on the seed_val
        # "agent_start": (x, y) to be set in the MyCustomGrid class depending on the seed_val
    }
    
# Modify an existing configuration, adding randomness.
def edit_config(old_config):
    new_config = dict(old_config)
    #new_config["num_blocks"] = max(0, old_config["num_blocks"] + np.random.choice([-2, -1, 1, 2]))
    offset = np.random.choice([-10, -5, 5, 10])
    new_config["seed_val"] = (old_config["seed_val"] + offset) % 1000000
    return new_config

# Calculate the regret as the difference between the teacher's performance, this is not the
# actual method used in the ACCEL paper.
def calculate_regret(config, student_model,teacher_model, max_steps=50):
    """
    Calculate regret as the difference between the teacher's performance
    and the student's performance on the same level.
    """
    env = MyCustomGrid(config)
    # Teacher rollout 
    obs, _ = env.reset()
    teacher_total_reward = 0
    for _ in range(max_steps):
        action, _ = teacher_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, _ = env.step(action)
        teacher_total_reward += reward
        if done or truncated:
            break

    # Student rollout
    obs, _ = env.reset()
    student_total_reward = 0
    for _ in range(max_steps):
        action, _ = student_model.predict(obs, deterministic=True)
        obs, reward, done, truncated, _ = env.step(action)
        student_total_reward += reward
        if done or truncated:
            break

    return max(0, teacher_total_reward - student_total_reward)

# Calculate regret using Generalized Advantage Estimation (GAE) with Stable-Baselines3's PPO model.
# PLR approximates regret using a score function such as the positive value loss.
def calculate_regret_gae(env, model, max_steps, gamma, lam):
    """
    Calculate regret using Generalized Advantage Estimation (GAE)
    with Stable-Baselines3's PPO model.
    """
    obs, _ = env.reset()
    regrets = []
    rewards = []
    dones = []
    values = []

    for t in range(max_steps):
        # Use the model's policy to get the value and action
        action, _ = model.predict(obs, deterministic=True)
        obs = torch.as_tensor(obs).float()
        value_t = model.policy.predict_values(obs).item()  # Access value function via model's policy

        obs, reward, done, truncated, _ = env.step(action)
        rewards.append(reward)
        values.append(value_t)
        dones.append(done)

        if done or truncated:
            break

    # Add value of the terminal state (0 if done/truncated)
    terminal_value = 0.0 if done else model.policy.predict_values(obs).item()
    values.append(terminal_value)

    # Compute TD-errors and GAE-like regret score
    for t in range(len(rewards)):
        delta_t = rewards[t] + gamma * values[t + 1] * (1 - dones[t]) - values[t]
        discounted_error = (gamma * lam) ** t * delta_t
        regrets.append(max(0, discounted_error))

    # Return the maximum positive regret score
    return max(regrets)



def initialize_ppo(env, learning_rate=1e-4):
    return PPO(
        "MlpPolicy",                    # Multi-layer perceptron policy
        env,                            # environment to learn from
        verbose=0,                      # Display training output
        n_steps=256,                    # Number of steps to run for each environment per update
        batch_size=64,                  # Minibatch size for each gradient update
        learning_rate=learning_rate
    )


# ====================================================
# 4. Main ACCEL Loop
# ====================================================

def main_accel_demo(total_iterations, replay_prob, train_steps, num_pretrain_levels,
                    level_buffer_size, initial_fill_size, pretrain_times, grid_size):
    
    
    # Create a level buffer to store generated levels and their scores
    level_buffer = LevelBuffer(max_size=level_buffer_size)
    iteration_regrets = []
    
    #Create a dummy environment to initialize the model
    dummy_env = MyCustomGrid(random_config(grid_size))

    # Initialize teacher and student models with PPO
    print("Initializing teacher and student models PPO...")
    teacher_model = initialize_ppo(dummy_env)
    student_model = initialize_ppo(dummy_env)
    

    # Pretrain teacher on a set of solvable random levels
    print("\nPHASE 1 - Pretraining of the teacher model...")
    for i in range(num_pretrain_levels):
        # Create a random config and environment from it
        cfg = random_config(grid_size)
        env = MyCustomGrid(cfg) # render_mode='human' to visualize the environment
        
        # Set the environment for the teacher model, train and evaluate it
        print(f"Pretraining teacher on config: {cfg} for {pretrain_times} times")
        teacher_model.set_env(env)
        teacher_model.learn(total_timesteps=pretrain_times) # Number of times the model is trained in the environment
        mean_reward, std_reward = evaluate_policy(teacher_model, env, n_eval_episodes=10, render=False)
        
        if mean_reward == 0:         # If the reward is 0, increase the loop counter to train the teacher on a different level
            print("Reward is 0, training the teacher on a different level\n")
            i -= 1
            continue
        else:
            print(f"Mean reward: {mean_reward}, Std reward: {std_reward}\n")
        


    # Populate buffer with initial levels
    print(f"Populating buffer with {initial_fill_size} initial levels and regrets...")
    for _ in range(initial_fill_size):
        cfg = random_config(grid_size)
        regret = calculate_regret(cfg, student_model, teacher_model)
        level_buffer.add(cfg, regret)

    for iteration in range(total_iterations):
        print(f"\n=== ITERATION {iteration + 1}/{total_iterations} ===")
        use_replay = np.random.rand() < replay_prob
        # Generates new random levels if you don't use replay
        if not use_replay or len(level_buffer.data) == 0:
            cfg = random_config(grid_size)
            regret = calculate_regret(cfg, student_model, teacher_model)
            #regret = calculate_regret_gae(MyCustomGrid(cfg), student_model, max_steps=50, gamma=0.99, lam=0.95)
            level_buffer.add(cfg, regret)
            print(f"  Sampled new config, regret={regret:.3f}")
        else:
            # Replays an existing layer, edits it, and evaluates the new layer
            old_cfg = level_buffer.sample_config()
            env = MyCustomGrid(old_cfg)
            student_model.set_env(env)
            student_model.learn(total_timesteps=train_steps)

            new_cfg = edit_config(old_cfg)
            regret = calculate_regret(new_cfg, student_model, teacher_model)
            #regret = calculate_regret_gae(MyCustomGrid(new_cfg), student_model, max_steps=50, gamma=0.99, lam=0.95)
            level_buffer.add(new_cfg, regret)
            print(f"  Replayed + mutated config, regret={regret:.3f}")
        
        iteration_regrets.append(regret)

    # Visualize progress of the regret over iterations.
    plt.figure(figsize=(8, 4))
    plt.plot(iteration_regrets, marker='o')
    plt.xlabel("Iteration")
    plt.ylabel("Regret")
    plt.title("Regret Progress during ACCEL Training")
    plt.grid(True)
    plt.show()
    
    print("\nDone. Final buffer size:", len(level_buffer.data))
    print("Top-5 hardest levels (config, regret):")
    level_buffer.data.sort(key=lambda x: x[1], reverse=True)
    for i, (cfg, sc) in enumerate(level_buffer.data[:5]):
        print(f"{i + 1}. regret={sc:.3f}, config={cfg}")
    
    

if __name__ == "__main__":
    
    config = {
        "num_pretrain_levels": 5,
        "pretrain_times": 100,
        "grid_size": 5,
        
        "total_iterations": 30,
        "replay_prob": 0.7,
        "train_steps": 500,
        "level_buffer_size": 50,
        "initial_fill_size": 25
    }
    
    print("Running ACCEL with config:")
    print(config, "\n")
    
    main_accel_demo(**config)


Running ACCEL with config:
{'num_pretrain_levels': 5, 'pretrain_times': 100, 'grid_size': 5, 'total_iterations': 30, 'replay_prob': 0.7, 'train_steps': 500, 'level_buffer_size': 50, 'initial_fill_size': 25} 

Initializing teacher and student models PPO...

PHASE 1 - Pretraining of the teacher model...
Pretraining teacher on config: {'width': 5, 'height': 5, 'seed_val': 650211} for 100 times


c:\Users\noost\anaconda3\envs\accel\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Reward is 0, training the teacher on a different level

Pretraining teacher on config: {'width': 5, 'height': 5, 'seed_val': 880477} for 100 times
Mean reward: 0.09819999933242798, Std reward: 0.2945999979972839

Pretraining teacher on config: {'width': 5, 'height': 5, 'seed_val': 9302} for 100 times
Mean reward: 0.09819999933242798, Std reward: 0.2945999979972839

Pretraining teacher on config: {'width': 5, 'height': 5, 'seed_val': 182254} for 100 times
Reward is 0, training the teacher on a different level

Pretraining teacher on config: {'width': 5, 'height': 5, 'seed_val': 104210} for 100 times
Reward is 0, training the teacher on a different level

Populating buffer with 25 initial levels and regrets...

=== ITERATION 1/30 ===


RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x15 and 75x64)